# **Checking runtime type**

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# **Cloning repo**

In [2]:
!git clone https://github.com/btoure98/Deep-manga-text-segmentation.git

Cloning into 'Deep-manga-text-segmentation'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 64 (delta 27), reused 57 (delta 20), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [3]:
%cd Deep-manga-text-segmentation

/content/Deep-manga-text-segmentation


In [4]:
!git pull

Already up to date.


**Mounting gdrive**

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


**Setup config**

In [11]:
image = "../gdrive/MyDrive/one_piece_test.png"
masks_destination = "../gdrive/MyDrive/output_onepiece.png"

models = "../gdrive/MyDrive/models/"
model_name = "simple_Unet34.pth"
IMG_HEIGHT = 1024
IMG_WIDTH = 1024


Dependencies install

In [7]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 88 kB 5.3 MB/s 
     |████████████████████████████████| 376 kB 16.7 MB/s 
     |████████████████████████████████| 58 kB 8.7 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=abc008656e8a2cc0b6786d0db4f3451ae2a260bb4d7f925fecfb6ededb6d4359
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=d3d20e4f554f0efd6ce96571288cecfc47be4ee6a5caaa9bb8a37be7e3e11cba
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


IMPORTS

In [8]:
import os
import segmentation_models_pytorch as smp
import torch

import cv2
from torch.utils.data import DataLoader
import torchvision.transforms as T

In [9]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Evalution**

In [12]:
import matplotlib.pyplot as plt
from torchvision.utils import save_image

# Create the model
model = torch.load(models + model_name)
model.eval()
# Create the preprocessing transformation here
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((IMG_HEIGHT, IMG_WIDTH)),
    T.ToTensor()])

transform_mask = T.Compose([
    T.ToPILImage(),
    T.Grayscale(),
    T.Resize((IMG_HEIGHT, IMG_WIDTH)),
    T.ToTensor()])

# load your image(s)
img = cv2.imread(image)

# Transform
input = transform(img).to(DEVICE)

# unsqueeze batch dimension, in case you are dealing with a single image
input = input.unsqueeze(0)

# Get prediction
output = model(input).squeeze(0)
#cv2.imwrite(masks_destination + img_name, output.squeeze(0).detach().cpu().numpy())
save_image(output, masks_destination)